In [2]:
!git clone https://github.com/cr-nlp/project1-2023.git

import urllib.request as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
import nltk
nltk.download('stopwords')
nltk.download('punkt')


def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc






fatal: destination path 'project1-2023' already exists and is not an empty directory.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def text2TokenList(text):
	stopword = stopwords.words('english')
	#print("LEN DE STOPWORD=",len(stopword))
	word_tokens = word_tokenize(text.lower())
	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>2]
	return word_tokens_without_stops

def run_bm25_only(startDoc,endDoc):

	dicDoc, dicReq, dicReqDoc = loadNFCorpus()

	docsToKeep=[]
	reqsToKeep=[]
	dicReqDocToKeep=defaultdict(dict)

	#150
	ndcgTop=10
	#print("ndcgTop=",ndcgTop,"nbDocsToKeep=",nbDocsToKeep)

	i=startDoc
	for reqId in dicReqDoc:
		if i > (endDoc - startDoc) :  #nbDocsToKeep:
			break
		for docId in dicReqDoc[reqId]:
			dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
			docsToKeep.append(docId)
			i = i + 1
		reqsToKeep.append(reqId)
	docsToKeep = list(set(docsToKeep))

	#"""
	allVocab ={}
	for k in docsToKeep:
		docTokenList = text2TokenList(dicDoc[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListDoc = list(allVocab)
	#print("doc vocab=",len(allVocabListDoc))
	allVocab ={}
	for k in reqsToKeep:
		docTokenList = text2TokenList(dicReq[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListReq = list(allVocab)

	from rank_bm25 import BM25Okapi

	corpusDocTokenList = []
	corpusReqTokenList = {}
	corpusDocName=[]
	corpusDicoDocName={}
	i = 0
	for k in docsToKeep:
		docTokenList = text2TokenList(dicDoc[k])
		corpusDocTokenList.append(docTokenList)
		corpusDocName.append(k)
		corpusDicoDocName[k] = i
		i = i + 1

	#print("reqs...")
	corpusReqName=[]
	corpusDicoReqName={}
	i = 0
	for k in reqsToKeep:
		reqTokenList = text2TokenList(dicReq[k])
		corpusReqTokenList[k] = reqTokenList
		corpusReqName.append(k)
		corpusDicoReqName[k] = i
		i = i + 1

	#print("bm25 doc indexing...")
	bm25 = BM25Okapi(corpusDocTokenList)

	ndcgCumul=0
	corpusReqVec={}
	ndcgBM25Cumul=0
	nbReq=0

	from sklearn.metrics import ndcg_score
	for req in corpusReqTokenList:
		j=0
		reqTokenList = corpusReqTokenList[req]
		doc_scores = bm25.get_scores(reqTokenList)
		trueDocs = np.zeros(len(corpusDocTokenList))

		for docId in corpusDicoDocName:
			if req in dicReqDocToKeep:
				if docId in dicReqDocToKeep[req]:
					#get position docId
					posDocId = corpusDicoDocName[docId]
					trueDocs[posDocId] = dicReqDocToKeep[req][docId]
					#print("TOKEEP=",docId)
					#print(trueDocs)
		ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
		nbReq = nbReq + 1
	ndcgBM25Cumul = ndcgBM25Cumul / nbReq
	print("ndcg bm25=",ndcgBM25Cumul)
	return ndcgBM25Cumul

nb_docs = 3192 #all docs
#nb_docs = 150 #for tests
run_bm25_only(0,nb_docs)

In [11]:
from gensim.models import KeyedVectors

# Load BioWordVec embeddings
bio_word_vec_path = 'C:/Users/alexa/Downloads/BioWordVec_PubMed_MIMICIII_d200.vec.bin'  # Update with your file path
bio_word_vec = KeyedVectors.load_word2vec_format(bio_word_vec_path, binary=True)

MemoryError: Unable to allocate 12.3 GiB for an array with shape (16545452, 200) and data type float32

In [12]:
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1
            feature_vector = np.add(feature_vector, model[word])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector

def averaged_doc_representation(corpus, model):
    vocabulary = set(model.index_to_key)
    num_features = model.vector_size
    doc_feature_vectors = [average_word_vectors(doc, model, vocabulary, num_features) for doc in corpus]
    return doc_feature_vectors

In [3]:
from collections import defaultdict
import numpy as np
from gensim.models import KeyedVectors
from sklearn.metrics import ndcg_score
from sklearn.metrics.pairwise import cosine_similarity

# Assuming functions like loadNFCorpus, text2TokenList exist as in your original code



def rank_documents(query_vector, doc_vectors):
    similarities = cosine_similarity([query_vector], doc_vectors)
    return similarities[0]

def run_with_word2vec_embeddings(startDoc, endDoc):
    dicDoc, dicReq, dicReqDoc = loadNFCorpus()

    docsToKeep, reqsToKeep, dicReqDocToKeep = select_documents(dicReqDoc, startDoc, endDoc)
    
    # Prepare the document vectors
    doc_vectors = averaged_doc_representation([text2TokenList(dicDoc[doc]) for doc in docsToKeep], bio_word_vec)

    ndcgCumul = 0
    nbReq = 0
    ndcgTop = 10  # Set the top N for NDCG calculation

    for req in reqsToKeep:
        reqTokenList = text2TokenList(dicReq[req])
        query_vector = average_word_vectors(reqTokenList, bio_word_vec, set(bio_word_vec.index_to_key), bio_word_vec.vector_size)
        trueRelevance = get_true_relevance(req, dicReqDocToKeep, docsToKeep)

        predicted_scores = rank_documents(query_vector, doc_vectors)
        ndcg_value = ndcg_score([trueRelevance], [predicted_scores], k=ndcgTop)

        ndcgCumul += ndcg_value
        nbReq += 1

    average_ndcg = ndcgCumul / nbReq
    print("Average NDCG with Word2Vec Embeddings:", average_ndcg)
    return average_ndcg

def get_true_relevance(req, dicReqDocToKeep, docsToKeep):
    true_relevance = [0] * len(docsToKeep)
    if req in dicReqDocToKeep:
        for docId, relevance in dicReqDocToKeep[req].items():
            if docId in docsToKeep:
                index = docsToKeep.index(docId)
                true_relevance[index] = relevance
    return true_relevance



# Example usage
nb_docs = 3192  # Use the full range or adjust as needed
run_with_word2vec_embeddings(0, nb_docs)


NameError: name 'text2TokenList' is not defined

In [1]:
from rank_bm25 import BM25Okapi
import numpy as np
from gensim.models import KeyedVectors
from sklearn.metrics import ndcg_score
from sklearn.metrics.pairwise import cosine_similarity
import urllib.request as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import pipeline
import string
import numpy as np
from collections import defaultdict
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')





def clean_sentence(sentence):

	stopword = stopwords.words('english')

	# Remove URLs
	sentence = re.sub(r'http\S+', '', sentence, flags=re.MULTILINE)
	sentence = re.sub(r'www\S+', '', sentence, flags=re.MULTILINE)

	# Remove all numbers
	sentence = re.sub(r'\d', '', sentence)

	# Remove punctuation except for certain cases
	sentence = re.sub(r'[^\w\s\-/]', '', sentence)

	word_tokens = word_tokenize(sentence.lower())
		
	lemmatizer = WordNetLemmatizer()
	normalised_tokens = [lemmatizer.lemmatize(token) for token in word_tokens]


	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>3]
	return word_tokens_without_stops


# we improved this by using a set
def generate_vocab(kept, loaded):

  vocab = set()

  for k in kept:
    vocab.update(clean_sentence(loaded[k]))

  return list(vocab)


# improved version with i and k by using enumerate by index tracking, this avoids using i += 1.
def generate_corpus(kept, loaded, corpus_token_list):
    corpus_name = []
    corpus_dic = {}

    for i, k in enumerate(kept):
        token_list = clean_sentence(loaded[k])

        if isinstance(corpus_token_list, dict):
          corpus_token_list[k] = (token_list)
        else:
          corpus_token_list.append(token_list)

        corpus_name.append(k)
        corpus_dic[k] = i

    return corpus_token_list, corpus_name, corpus_dic

def summarise_corpus(kept, loaded):
    summariser = pipeline('summarization')
    summaries = []

    for doc in kept:
        summaries.append(summariser(loaded[doc], min_length=10, max_length = 100, do_sample=False))
    
    return summaries

def normalise(score):
    return (score - np.mean(score)) / np.std(score)

# keep relevant: only keeps the relevant values together
def shorten_dictionary(doc, req, req_doc, keep_relevant, start, end):
  kept_doc = []
  kept_req = []
  kept_req_doc = defaultdict(dict)

  if(keep_relevant):
    req_doc = {outer_key: {inner_key: inner_value for inner_key, inner_value in inner_dict.items() if inner_value == 2}
                            for outer_key, inner_dict in req_doc.items()}

  count = start
  for req in req_doc:
    if count > (end - start):
      break
    for doc in req_doc[req]:
      kept_req_doc[req][doc] = req_doc[req][doc]
      kept_doc.append(doc)
      count += 1
    kept_req.append(req)
  kept_doc = list(set(kept_doc)) # we remove duplicates

  return kept_doc, kept_req, kept_req_doc




# Assuming functions like loadNFCorpus, text2TokenList exist as in your original code
# Load your BioWordVec embeddings
bio_word_vec_path = 'C:/Users/alexa/Downloads/BioWordVec_PubMed_MIMICIII_d200.vec.bin'  # Update with your file path
bio_word_vec = KeyedVectors.load_word2vec_format(bio_word_vec_path, binary=True)

!git clone https://github.com/cr-nlp/project1-2023.git


def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc

c:\Users\alexa\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
fatal: destination path 'project1-2023' already exists and is not an empty directory.


In [8]:
import re 

def normalize_scores(scores):
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    if std_score != 0:
        normalized_scores = (scores - mean_score) / std_score
    else:
        normalized_scores = scores - mean_score  # Avoid division by zero
    return normalized_scores


def select_documents(dicReqDoc, startDoc, endDoc):
    docsToKeep = []
    reqsToKeep = []
    dicReqDocToKeep = defaultdict(dict)

    i = 0
    for reqId, docs in dicReqDoc.items():
        if i > endDoc - startDoc:
            break
        for docId in docs:
            dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
            docsToKeep.append(docId)
            i += 1
        reqsToKeep.append(reqId)
    docsToKeep = list(set(docsToKeep))

    return docsToKeep, reqsToKeep, dicReqDocToKeep
# Function to compute average word vectors

def text2TokenList(text):
	stopword = stopwords.words('english')
	#print("LEN DE STOPWORD=",len(stopword))
	word_tokens = word_tokenize(text.lower())
	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>2]
	return word_tokens_without_stops

def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1
            feature_vector = np.add(feature_vector, model[word])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector

# Function to get averaged document representation
def averaged_doc_representation(corpus, model):
    vocabulary = set(model.index_to_key)
    num_features = model.vector_size
    doc_feature_vectors = [average_word_vectors(doc, model, vocabulary, num_features) for doc in corpus]
    return doc_feature_vectors

# Function to compute BM25 scores
def compute_bm25_scores(corpus, query):
    tokenized_corpus = [text2TokenList(doc) for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    
    query_tokens = clean_sentence(query)

    #query_tokens = text2TokenList(query)
    bm25_scores = bm25.get_scores(query_tokens)
    normalized_bm25_scores = normalize_scores(np.array(bm25_scores))
    return normalized_bm25_scores

# Function to compute cosine similarity scores
def compute_cosine_scores(docs, query, bio_word_vec):
    query_vector = average_word_vectors(text2TokenList(query), bio_word_vec, set(bio_word_vec.index_to_key), bio_word_vec.vector_size)
    doc_vectors = averaged_doc_representation([text2TokenList(doc) for doc in docs], bio_word_vec)
    cosine_scores = rank_documents(query_vector, doc_vectors)
    normalized_cosine_scores = normalize_scores(np.array(cosine_scores))
    return normalized_cosine_scores

# Function to compute hybrid score
def hybrid_score(bm25_scores, cosine_scores, alpha=0.7):
    combined_scores = alpha * bm25_scores + (1 - alpha) * cosine_scores
    return combined_scores

# Function to rank documents
def rank_documents(query_vector, doc_vectors):
    similarities = cosine_similarity([query_vector], doc_vectors)
    return similarities[0]

# Function to evaluate with NDCG
def evaluate_with_ndcg(reqs, docs, dicDoc,dicReq, dicReqDocToKeep, bm25, bio_word_vec, alpha=0.5, ndcgTop=5):
    ndcgCumul = 0
    nbReq = 0
    
    for req in reqs:
        req_text = dicReq[req]
        bm25_scores = compute_bm25_scores([dicDoc[doc] for doc in docs], req_text)
        cosine_scores = compute_cosine_scores([dicDoc[doc] for doc in docs], req_text, bio_word_vec)
        hybrid_scores = hybrid_score(bm25_scores, cosine_scores, alpha)

        trueRelevance = get_true_relevance(req, dicReqDocToKeep, docs)
        ndcg_value = ndcg_score([trueRelevance], [hybrid_scores], k=ndcgTop)
        ndcgCumul += ndcg_value
        nbReq += 1

    average_ndcg = ndcgCumul / nbReq
    return average_ndcg

# Function to get true relevance scores
def get_true_relevance(req, dicReqDocToKeep, docs):
    true_relevance = [0] * len(docs)
    if req in dicReqDocToKeep:
        for docId, relevance in dicReqDocToKeep[req].items():
            if docId in docs:
                index = docs.index(docId)
                true_relevance[index] = relevance
    return true_relevance

# Main function to run
def run_with_hybrid_method(startDoc, endDoc, alpha=0.5):
    dicDoc, dicReq, dicReqDoc = loadNFCorpus()
    docsToKeep, reqsToKeep, dicReqDocToKeep = select_documents(dicReqDoc, startDoc, endDoc)

    # Assuming you already have a BM25 model initialized
    bm25 = BM25Okapi([text2TokenList(dicDoc[doc]) for doc in docsToKeep])

    average_ndcg = evaluate_with_ndcg(reqsToKeep, docsToKeep,dicDoc, dicReq, dicReqDocToKeep, bm25, bio_word_vec, alpha=alpha)
    print("Average NDCG with Hybrid Method:", average_ndcg)
    return average_ndcg

# Example usage
nb_docs = 150 #Use the full range or adjust as needed
run_with_hybrid_method(0, nb_docs)


Average NDCG with Hybrid Method: 0.7595836394196158


0.7595836394196158